In [3]:
import geopandas as gpd

## Open GIS Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)
states_gdf = states_gdf[["STUSPS", "geometry"]]

## Open Shell Data

In [7]:
shell_gdf = gpd.read_file("data/Shell_Gas_Stations_Per_State.gpkg")
shell_gdf["shell_per_100k"] = shell_gdf["per_100k"].round(decimals=1)
shell_gdf["SHELLs"] = shell_gdf["COUNT"]
shell_gdf = shell_gdf[["STUSPS", "SHELLs", "shell_per_100k"]]

## Open Exxon Data

In [9]:
exxon_gdf = gpd.read_file("data/Exxon_Gas_Stations_Per_State.gpkg")
exxon_gdf["exxons_per_100k"] = exxon_gdf["per_100k"].round(decimals=1)
exxon_gdf["EXXONs"] = exxon_gdf["COUNT"]
exxon_gdf = exxon_gdf[["STUSPS", "EXXONs", "exxons_per_100k"]]

## Merge Data

In [11]:
exxon_states_gdf = states_gdf.merge(exxon_gdf, on="STUSPS", how="left")
exxon_shell_states_gdf = exxon_states_gdf.merge(shell_gdf, on="STUSPS", how="left")

In [12]:
exxon_shell_states_gdf = exxon_shell_states_gdf.fillna(0)

In [14]:
exxon_shell_states_gdf["SHELLs"] = exxon_shell_states_gdf["SHELLs"].astype(int)
exxon_shell_states_gdf["EXXONs"] = exxon_shell_states_gdf["EXXONs"].astype(int)

In [15]:
def more_stores(row) -> str:
    shell = row["SHELLs"]
    exxon = row["EXXONs"]
    if shell > exxon:
        return "More Shells"
    if shell < exxon:
        return "More Exxons"
    if shell == 0 and exxon == 0:
        return "No Locations"
    if shell == exxon:
        return "Same Amount of Chipotles & QDOBAs"
    else:
        return "Edge Case"

In [16]:
exxon_shell_states_gdf["label"] = exxon_shell_states_gdf.apply(
    lambda row: more_stores(row), axis=1
)

In [18]:
exxon_shell_states_gdf[exxon_shell_states_gdf["label"] == "Edge Case"]

,STUSPS,geometry,EXXONs,exxons_per_100k,SHELLs,shell_per_100k,label


In [19]:
exxon_shell_states_gdf = exxon_shell_states_gdf.to_crs(9311)
exxon_shell_states_gdf.to_file("data/Shell_Vs_Exxon_Per_State.gpkg")

In [29]:
exxon_shell_states_gdf.sort_values("exxons_per_100k", ascending=False).head(3)

,STUSPS,geometry,EXXONs,exxons_per_100k,SHELLs,shell_per_100k,label
30,VT,"POLYGON ((2075870.027 238820.542, 2075640.482 ...",87,13.4,45,6.9,More Exxons
31,MT,"POLYGON ((-1172889.072 508069.731, -1172583.63...",103,9.1,0,0.0,More Exxons
4,WV,"POLYGON ((1506482.53 -601393.056, 1506489.516 ...",159,9.0,17,1.0,More Exxons


In [30]:
exxon_shell_states_gdf.sort_values("shell_per_100k", ascending=False).head(3)

,STUSPS,geometry,EXXONs,exxons_per_100k,SHELLs,shell_per_100k,label
0,MS,"MULTIPOLYGON (((1109997.773 -1565578.875, 1111...",182,6.2,259,8.8,More Shells
23,TN,"POLYGON ((883547.508 -1059435.53, 883522.954 -...",409,5.7,609,8.5,More Shells
5,LA,"MULTIPOLYGON (((1079322.442 -1609024.878, 1079...",298,6.5,329,7.2,More Shells


In [31]:
exxon_shell_states_gdf.sort_values("SHELLs", ascending=False).head(3)

,STUSPS,geometry,EXXONs,exxons_per_100k,SHELLs,shell_per_100k,label
15,TX,"MULTIPOLYGON (((514197.103 -1677430.313, 51428...",1950,6.3,1682,5.5,More Exxons
16,CA,"MULTIPOLYGON (((-1717282.257 -1090154.595, -17...",565,1.4,1374,3.5,More Shells
29,IL,"POLYGON ((720088.004 -498699.252, 720164.325 -...",451,3.6,763,6.0,More Shells


In [32]:
exxon_shell_states_gdf.sort_values("EXXONs", ascending=False).head(3)

,STUSPS,geometry,EXXONs,exxons_per_100k,SHELLs,shell_per_100k,label
15,TX,"MULTIPOLYGON (((514197.103 -1677430.313, 51428...",1950,6.3,1682,5.5,More Exxons
9,FL,"MULTIPOLYGON (((1997726.172 -1929592.915, 1997...",1011,4.4,704,3.1,More Exxons
25,NY,"MULTIPOLYGON (((2282948.092 -21624.723, 228289...",759,3.8,338,1.7,More Exxons
